This section will be an introduction to the principles of Bayesian inference. Will cover Bayesian goals and terminology. Will introduce posterior inference via conjugate priors with a couple simple examples. Will stop short of MCMC methods. 

*Notes:*
- Could probably get some good use out of something like fantasy football point-per-player-per-game data. Could do conjugate priors, poisson-gamma distribution, unsupervised clustering...